# Checkersnet

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np


In [2]:
import sys
sys.path.append('/home/dawki/Projects/Checkersmate')
from checkersmate.game import Game

## Data

In [3]:
class CheckersDataset(Dataset):

    def __init__(self,files,records_per_file=60000,starting_file=0):
        self.files = files
        self.records_per_file = records_per_file
        self.current_file = starting_file
        data = np.loadtxt(f"data/input/result_{self.current_file}.csv",delimiter = ",",dtype=np.float32)
        self.x = torch.from_numpy(data[0:self.records_per_file,0:32])
        self.y = torch.from_numpy(data[0:self.records_per_file,[32]])

    def __getitem__(self, index):
        if(index>=self.files * self.records_per_file):
            raise IndexError(f"index out of range")
        start = self.current_file*self.records_per_file
        end = (self.current_file+1)*self.records_per_file
        if((start <= index) and index < end):
            return self.x[index-start], self.y[index-start]
        else:
            self.current_file = index//self.records_per_file
            start = self.current_file*self.records_per_file
            end = (self.current_file+1)*self.records_per_file

            data = np.loadtxt(f"data/input/result_{self.current_file}.csv",delimiter = ",",dtype=np.float32)
            self.x = torch.from_numpy(data[0:self.records_per_file,0:32])
            self.y = torch.from_numpy(data[0:self.records_per_file,[32]])

            return self.x[index-start], self.y[index-start]

    def __len__(self):
        return self.files * self.records_per_file

In [4]:
trainloader = DataLoader(CheckersDataset(10), batch_size=20000, shuffle=False, num_workers=2)

In [5]:
testloader = DataLoader(CheckersDataset(files=1,starting_file=10), batch_size=20000, shuffle=False, num_workers=2)

## Model

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(32, 50)
        self.fc2 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 25)
        self.fc4 = nn.Linear(25, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


net = Net()
print(net)

Net(
  (fc1): Linear(in_features=32, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=25, bias=True)
  (fc4): Linear(in_features=25, out_features=1, bias=True)
)


## Optimisation

In [6]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

## Training

In [11]:
def training_loop(epochs):
    print(f'Started Training - {epochs}')
    for epoch in range(epochs):  # loop over the dataset multiple times
        print(f"e: {epoch+1}")
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print(f'Finished Training - {epochs}')
    torch.save(net.state_dict(), f"./model/{epochs}.pth")


In [ ]:
for e in range(10):
    training_loop(2**(e))

## Testing

In [7]:
dataiter = iter(testloader)
boards, results = dataiter.next()

In [20]:
for e in range(5):
    net = Net()
    net.load_state_dict(torch.load(f"./model/{2**(e)}.pth"))
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            boards, results = data
            # calculate outputs by running images through the network
            outputs = net(boards)
            # the class with the highest energy is what we choose as prediction
            predicted = 2*((outputs>0).type(torch.int32)-0.5)
            total += results.size(0)
            correct += (predicted == results).sum().item()

            print(f'Accuracy of the network at {e} epochs: %d %%' % (100 * correct / total))

Accuracy of the network at 0 epochs: 51 %
Accuracy of the network at 0 epochs: 53 %
Accuracy of the network at 0 epochs: 53 %
Accuracy of the network at 1 epochs: 59 %
Accuracy of the network at 1 epochs: 60 %
Accuracy of the network at 1 epochs: 61 %
Accuracy of the network at 2 epochs: 62 %
Accuracy of the network at 2 epochs: 64 %
Accuracy of the network at 2 epochs: 65 %
Accuracy of the network at 3 epochs: 62 %
Accuracy of the network at 3 epochs: 64 %
Accuracy of the network at 3 epochs: 65 %
Accuracy of the network at 4 epochs: 63 %
Accuracy of the network at 4 epochs: 64 %
Accuracy of the network at 4 epochs: 65 %


In [29]:
otb = net(torch.Tensor([1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]))

In [30]:
otb[0]

tensor(-0.8826, grad_fn=<SelectBackward>)

In [24]:
data[0][0]

tensor([ 0.,  0.,  0.,  0.,  0.,  0., -2.,  0., -1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  2.,  0.,  0.])

## Testing with games

In [8]:
i = 101
b = boards[i].tolist()
r = results[i]

In [12]:
trials = {"-1":0,"0":0,"1":0}
for i in range(10000):
    g = Game(b, silent=True)
    trials[str(g.play())] += 1
    if (i%100==0): print(f"{i//100}%")

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [14]:
trials

{'-1': 532, '0': 497, '1': 8971}

## Play against bot

In [15]:
test_game = Game()

In [18]:
test_game.play()

 ___________Checkersmate____________
    red (x)'s turn 
  +---+---+---+---+---+---+---+---+
8 |   | o |   | o |   | o |   | o |
  |---+---+---+---+---+---+---+---|   
7 | o |   | o |   | o |   | o |   |
  |---+---+---+---+---+---+---+---|
6 |   | o |   | o |   | o |   | o |
  |---+---+---+---+---+---+---+---|   
5 |   |   |   |   |   |   |   |   |
  |---+---+---+---+---+---+---+---|
4 |   |   |   |   |   |   |   |   |
  |---+---+---+---+---+---+---+---|   
3 | x |   | x |   | x |   | x |   |
  |---+---+---+---+---+---+---+---|
2 |   | x |   | x |   | x |   | x |
  |---+---+---+---+---+---+---+---|
1 | x |   | x |   | x |   | x |   |
  +---+---+---+---+---+---+---+---+
    A   B   C   D   E   F   G   H
 ___________Checkersmate____________
    white (o)'s turn 
  +---+---+---+---+---+---+---+---+
8 |   | o |   | o |   | o |   | o |
  |---+---+---+---+---+---+---+---|   
7 | o |   | o |   | o |   | o |   |
  |---+---+---+---+---+---+---+---|
6 |   | o |   | o |   | o |   | o |
  |---+---

1

## Misc

In [22]:
outputs = net(boards)
preds = 2*((outputs>0).type(torch.int32)-0.5)

In [48]:
preds = 2*((outputs>0).type(torch.int32)-0.5)

In [51]:
preds[0:5]

tensor([[-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.]])

In [43]:
results[0:5]

tensor([[ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.]])

In [52]:
(preds[0:5] == results[0:5]).sum().item()

2

In [15]:
for data in testloader:
    print(len(data))
    print(data[0].shape)
    print(data[1].shape)

2
torch.Size([20000, 32])
torch.Size([20000, 1])
2
torch.Size([20000, 32])
torch.Size([20000, 1])
2
torch.Size([20000, 32])
torch.Size([20000, 1])


In [54]:
total

60000